# Auto Annotations


In [1]:
cd ..

/Users/yanndubois/Desktop/GitHub/alpaca_farm


In [2]:
from alpaca_farm.utils import jload
from alpaca_farm.auto_annotations.pairwise_annotators import PairwiseAutoAnnotator, SinglePairwiseAutoAnnotator

The key object that you need for automatic annotations (both for training or for eval) is our `PairwiseAutoAnnotator`. By default the annotator is the pool from Alpaca Farm, for simplicity let's use a single annotator `test` for now.

For details about annotators including how to extend them, refer to the [Configuring annotators](#Configuring-annotators) section of this notebook.

In [9]:
annotator = PairwiseAutoAnnotator(annotators_config="annotators/test/configs.yaml")

Now let's annotate some pairwise preference. All we need is some data. The annotator takes in either list of dictionaries, or pandas dataframes. The keys of dictionaries (or columns in dataframe) need to always contain `instruction` and `input` which defines the instruction. The annotator also needs a pair of outputs to compare. We allow different format for the outputs, which are useful in different contexts as we will see below.

## Evaluation
For evaluation we need two components:
- outputs from the model we want to compare
- outputs on the same examples from the baseline model

Here let us compare a few examples from our simulated RLHF model and 003. We will thus give to the annotate two list of dictionaries, one for each model. Both need to contain an `output` column. Let us load such data. 

In [10]:
outputs_baseline = jload("examples/data/outputs_baseline.json")
print("Example of baseline output:\n")
outputs_baseline[-1:]

Example of baseline output:



[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]"}]

In [11]:
outputs_rlhf = jload("examples/data/outputs_rlhf.json")
print("Example of rlhf output:\n")
outputs_rlhf[-1:]

Example of rlhf output:



[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output': "Dear Friends,\n\nI'm planning a dinner party this Friday and I'd love for you to join me! It's going to be a casual night of great food, interesting conversations, and lots of laughs.\n\nI'm looking forward to seeing you there. Let me know if you have any dietary restrictions or if you have any questions.\n\nSincerely,\n[Your Name]"}]

The annotator's function of interest when we have two sequences of outputs is `annotate_head2head`.

In [12]:
annotated = annotator.annotate_head2head(outputs_1=outputs_baseline, outputs_2=outputs_rlhf)

prompt_batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.18s/it]


In [13]:
annotated[-1:]

[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output_1': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]",
  'output_2': "Dear Friends,\n\nI'm planning a dinner party this Friday and I'd love for you to join me! It's going to be a casual night of great food, interesting conversations, and lots of laughs.\n\nI'm looking forward to seeing you there. Let me know if you have any dietary restrictions or if you have any questions.\n\nSincerely,\n[Your Name]",
  'annotator': 'davinci003_3',
  'preference': 2.0}]

The annotation is then stored in `preference` and indicates the index of the prefered output. We see here that the auto annotator preferred the RLHF output `output_2`.

Note that in case you already have a sequence of dictionaries that contain both outputs as `output_1` and `output_2`, then the desired function for annotations is `annotate_pairs`. Under the hood `annotate_head2head` simply merges both sequences based on inputs and instructions and then calls  `annotate_pairs`. 


## Training

## Going further

### Cost & time efficiency: avoiding duplicate annotation
Often time you will find yourself reannotating examples that you already annotated. This is particularly true if you sample many times from the same model (eg to get pairwise preferences for training or to evaluate best-of-n) or if the instructions you are considering require short outputs => many models might give the same exact answer.

This means that you have to spend unecessary money and time. Thankfully `PairwiseAutoAnnotator` stores previous annotations and will reuse those. For example, let us reannotate the previous evaluation

In [9]:
annotated = annotator.annotate_head2head(outputs_1=outputs_baseline, outputs_2=outputs_rlhf)

In [10]:
annotated[-1:]

[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output_1': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]",
  'output_2': "Dear Friends,\n\nI'm planning a dinner party this Friday and I'd love for you to join me! It's going to be a casual night of great food, interesting conversations, and lots of laughs.\n\nI'm looking forward to seeing you there. Let me know if you have any dietary restrictions or if you have any questions.\n\nSincerely,\n[Your Name]",
  'annotator': 'chatgpt_4',
  'preference': 2.0}]

We now get the annotations without actually having to reannotate any example.
By default, this the previous annotations are only stored in memory. If you want those to be stateful, you need to either give `saving_path` to the constructor of `PairwiseAutoAnnotator` (automatically save and loads) or manually call `save` and `load`.
Here's an example

In [15]:
saving_path = "examples/data/annotations.json"
annotator.save(saving_path) # manually save the previous annotation
new_annotator = PairwiseAutoAnnotator(#annotators_config="annotators/test/configs.yaml",
                                     saving_path=saving_path # will automatically load and save future annotations
                                        )

In [16]:
annotated = new_annotator.annotate_head2head(outputs_1=outputs_baseline, outputs_2=outputs_rlhf)
annotated[-1:]

[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output_1': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]",
  'output_2': "Dear Friends,\n\nI'm planning a dinner party this Friday and I'd love for you to join me! It's going to be a casual night of great food, interesting conversations, and lots of laughs.\n\nI'm looking forward to seeing you there. Let me know if you have any dietary restrictions or if you have any questions.\n\nSincerely,\n[Your Name]",
  'annotator': 'chatgpt_4',
  'preference': 2.0}]

We suggest that you simply set `saving_path=...` when initializing the annotator and you will never have to deal with manual saving and loading.

### Configuring annotators

The most important argument to `PairwiseAutoAnnotator` is `annotators_config` which defines the pool of annotators. We provide the following options out of the box:
- `annotators_config="annotators/annotator_pool_v0/configs.yaml"`: ApacaFarm's default annotator pool
- `annotators_config="annotators/greedy_gpt4/configs.yaml"`: Greedy GPT4 annotator
- `annotators_config="annotators/test/configs.yaml"`: a faster text-davinci-003 annotator useful for testing

Here's the desciption of `annotators_config` from the docstring:
```
annotators_config : Path or list of dict
    A dictionary or path to a yaml file containing the configuration for the pool of annotators. The keys in the
        fist dictionary should be the annotator's name, and the value should be a dictionary of the annotator's
        configuration which should have the following keys:
        - prompt_templates (dict): a dictionary of prompts or path to the prompts.
        - fn_decoder (str): function in `alpaca_farm.auto_annotations.pairwise_annotators.decoders.py` for completions.
        - decoder_kwargs (dict): kwargs for fn_decode. E.g. model_name, max_completions_tokens
        - other kwargs to `SinglePairwiseAutoAnnotator`
```

And here is config `"annotators/test/configs.yaml"` of the annotator we used above. 

In [20]:
!cat src/alpaca_farm/auto_annotations/annotators/test/configs.yaml

davinci003_3 : # text-davinci-003_v1_b5-pairwise_temp=1.0
  prompt_templates:
    with_inputs: "annotators/annotator_pool_v0/text_b5_with_inputs.txt"
    without_inputs: "annotators/annotator_pool_v0/text_b5_without_inputs.txt"
  fn_decoder: "openai_completions"
  decoder_kwargs:
    model_name: "text-davinci-003"
    max_tokens: 200
    temperature: 0.0
  outputs_to_match:
    1: '\n\(a\)'
    2: '\n\(b\)'
  batch_size: 5

For more information see the docstrings of the following classes (eg uncomment and run the following)

In [15]:
# from alpaca_farm.auto_annotations.pairwise_annotators import SinglePairwiseAutoAnnotator
# PairwiseAutoAnnotator?

In [27]:
# SinglePairwiseAutoAnnotator?

If you want to make a new annotator, change the prompt and or the configs above. See `annotators/annotator_pool_v0/...txt` for the actual prompts we used. To see the prompt used above (which is part of our pool of prompts) uncomment and run the following line.

In [24]:
#!cat src/alpaca_farm/auto_annotations/annotators/annotator_pool_v0/text_b5_without_inputs.txt